In [1]:
# Dependencies
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
import seaborn as sns

edu_dict = {
    1.0 : "Less than 9th grade",
    2.0 : "9-11th grade",
    3.0 : "HS grad/GED",
    4.0 : "Some college", 
    5.0 : "College grad+",
    7.0 : "Other",
    9.0 : "Other"
}
DF["Educational Attainment"] = DF["Educational Attainment"].map(edu_dict)
DF_clean.head()

---------------------
race_dict = {
    1.0 : "Mex-Am",
    2.0 : "Other Hisp",
    3.0 : "White",
    4.0 : "Black", 
    6.0 : "Asian",
    7.0 : "Other/Multi",
    
}
Race_update["Race"] = Race_update["Race"].map(race_dict)
Race_update.head()

Gender_dic = {1.0 : "Male",2.0:"Female"}

Gender_Dataset["Gender"] = Gender_Dataset["Gender"].map(Gender_dic)
Gender_Dataset.head()